In [ ]:
#|default_exp autoencoders
#|export

import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
#|export

class VariationalAutoencoder(nn.Module):
    def __init__(self, input_size:int, sigmoid_mask: torch.Tensor):
        '''
        Variational Autoencoder for data compression and reconstruction.

        Args:
            input_size (int): The number of input features.
            sigmoid_mask (torch.Tensor): Boolean mask indicating which input features should have a sigmoid activation applied in the output layer (e.g., for binary or categorical features). Should be a 1D tensor of length input_size.
        '''
        super(VariationalAutoencoder, self).__init__()

        #Stores key model parameters
        self.input_size = input_size

        hidden_size_1 = 64
        hidden_size_2 = 32
        latent_size = 16
        
        # Encoder architecture
        self.encoder = nn.Sequential(
            nn.Linear(input_size, hidden_size_1),
            nn.SELU(),
            nn.Linear(hidden_size_1, hidden_size_2), 
            nn.SELU(),
        )
        
        # Latent space parameters
        self.fc_mean = nn.Linear(hidden_size_2, latent_size)  # Mean of latent distribution
        self.fc_log_variance = nn.Linear(hidden_size_2, latent_size)  # Log variance of latent distribution
        
        # Decoder architecture
        self.decoder = nn.Sequential(
            nn.Linear(latent_size, hidden_size_2),
            nn.SELU(),
            nn.Dropout(0.1),
            nn.Linear(hidden_size_2, hidden_size_1),
            nn.SELU(),
            nn.Linear(hidden_size_1, input_size)
        )

        self.register_buffer('sigmoid_mask', sigmoid_mask.unsqueeze(0))#Only values between 0 and 1 have a sigmoid function applied to them at the end
        
    def encode(self, x:torch.Tensor)->tuple[torch.Tensor, torch.Tensor]:
        '''
        Encodes the input data into the parameters of the latent space distribution (mean and log-variance).

        Args:
            x (torch.Tensor): Input tensor of shape [batch_size, input_size].

        Returns:
            tuple[torch.Tensor, torch.Tensor]:
                - mean (torch.Tensor): Mean of the latent Gaussian distribution, shape [batch_size, latent_size].
                - log_variance (torch.Tensor): Log-variance of the latent Gaussian distribution, shape [batch_size, latent_size].
        '''

        # Generate latent space parameters
        hidden = self.encoder(x)
        mean = self.fc_mean(hidden)
        log_variance = self.fc_log_variance(hidden)
        return mean, log_variance
    
    def reparameterise(self, mean:torch.Tensor, log_variance:torch.Tensor)->torch.Tensor:
        '''
        Applies the reparameterisation to allow sampling from the latent space, enabling backpropagation through stochastic nodes.

        Args:
            mean (torch.Tensor): Mean of the latent Gaussian distribution, shape [batch_size, latent_size].
            log_variance (torch.Tensor): Log-variance of the latent Gaussian distribution, shape [batch_size, latent_size].

        Returns:
            torch.Tensor: Sampled latent vector, shape [batch_size, latent_size].
        '''
        log_variance = F.softplus(log_variance) + 1e-6 

        std = torch.exp(0.5 * log_variance)
        eps = torch.randn_like(std)  # Random noise from standard normal
        return mean + eps * std
    
    def decode(self, latent_vector:torch.Tensor)->torch.Tensor:
        '''
        Decodes a latent vector back into the input feature space.

        Args:
            latent_vector (torch.Tensor): Latent space representation, shape [batch_size, latent_size].

        Returns:
            torch.Tensor: Reconstructed input, shape [batch_size, input_size].
        '''
        return self.decoder(latent_vector)
    
    def forward(self, x:torch.Tensor)->tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        '''
        Runs a forward pass through the VAE:
        
        - Encodes the input
        - Decodes from the latent mean
        - Applies the appropriate output activations.

        Args:
            x (torch.Tensor): Input tensor of shape [batch_size, input_size].

        Returns:
            tuple[torch.Tensor, torch.Tensor]:
                - reconstruction (torch.Tensor): The reconstructed input, with sigmoid applied to features as specified by sigmoid_mask.
                - mean (torch.Tensor): The mean of the latent distribution for each input in the batch.
        '''
        mean, _ = self.encode(x)
        #latent_vector = self.reparameterise(mean, log_variance)#Reparmatisation dropped to avoid latent space collapse
        raw_reconstruction = self.decode(mean)

        reconstruction = torch.where(
            self.sigmoid_mask,
            torch.sigmoid(raw_reconstruction),
            raw_reconstruction
            )

        return reconstruction, mean
